In [39]:
import numpy as np
import pandas as pd
import subprocess
import os
import shutil
import warnings
import matplotlib
import matplotlib.pyplot as plt
#%matplotlib inline
warnings.filterwarnings('ignore')

exp ='adaptation'
subj_list=["docnet1001", "docnet1002"] #subs to analyze
loc_suf = ["_all"] #runs to pull ROIs from
exp_suf = [""]
rois=["V3ab", "PPC", "APC", "V4", "LO", "PFS"] #Rois
loc_cope=[1, 1, 1, 2, 2, 2] #copes for localizer runs; corresponding numerically to each roi
exp_cope=[1, 2, 3, 4, 5, 6] #experimental copes to test in each ROI

bin_size=100
cond=["AC", "AD", "AP", "RC", "RD", "RP"]
cond_name = ["Adapt_Complete", "Adapt_Deleted", "Adapt_Perturbed",\
             "Release_Complete", "Release_Deleted", "Release_Perturbed"]

bool_extract_data = True
bool_calc_act = False
bool_calc_mvpa = False


exp_dir = "/lab_data/behrmannlab/vlad/docnet"

Function to extract functional localizer data from each ROI and parameter estimate from each mask

In [22]:
def extract_data(sub_dir, results_dir, lr, rr):
    #check if ROI exists
    for sf_loc in loc_suf: #loop across run type for the masks
        roi_nifti = f'{sub_dir}/rois/{lr}{rois[rr]}{sf_loc}.nii.gz'
        if os.path.exists(roi_nifti):

            for sf_exp in exp_suf: #loop across the run types for the experimental copes
                                
                for ec in range(0,len(exp_cope)):
                    cope_nifti = f"{sub_dir}/fsl/{exp}/HighLevel{sf_exp}.gfeat/cope{exp_cope[ec]}.feat/stats/zstat1.nii.gz"
                    out = f'{results_dir}/{lr}{rois[rr]}{sf_loc}{sf_exp}_{cond[ec]}'
                    #extract_data(roi_nifti, cope_nifti, f'{results_dir}/{lr}{rois[rr]}{sf_loc}_localizer.nii.gz')

                    bash_cmd  = f'fslmeants -i {cope_nifti} -m {roi_nifti} -o {out}.txt --showall --transpose'
                    #print(bash_cmd)
                    bash_out = subprocess.run(bash_cmd.split(),check=True, capture_output=True, text=True)
                    #print(bash_out.stdout)


Function to load localizer and test data, append, average together, and plot

In [32]:
def calc_activation(lr, rr, bin_size):
    for sf_loc in loc_suf: #loop across run type for the masks
        #define and read localzier files
        loc_file = f'{roi_dir}/data/{lr}{rois[rr]}{sf_loc}.txt'

        loc_df = pd.read_csv(loc_file, sep="  ", header=None, names = ["x", "y", "z", "loc"])

        for sf_exp in exp_suf: #loop across experimental types
            for cc in cond:
                #define exp file
                exp_file = f'{results_dir}/{lr}{rois[rr]}{sf_loc}{sf_exp}{cc}.txt'

                #load each file
                exp_df = pd.read_csv(exp_file, sep="  ", header=None, names = ["x", "y", "z", cc])

                #Append it to the localizer data
                loc_df = loc_df.join(exp_df[cc])


            #sort file by localizer functional value (high to low)
            loc_df = loc_df.sort_values(by =['loc'], ascending=False)

            #subsample dfs using a rolling average
            loc_df = loc_df.rolling(bin_size, win_type='triang').mean()
            loc_df = loc_df.dropna()

            #Reset the index on both
            loc_df= loc_df.reset_index(drop=True)

            return loc_df


Calculate within- between- Haxby-style MVPA

In [9]:
def calc_mvpa(lr, rr, bin_size):
    #define and read localzier files
    loc_file = f'{results_dir}/{lr}{rois[rr]}_all_localizer.txt'

    loc_df = pd.read_csv(loc_file, sep="  ", header=None, names = ["x", "y", "z", "loc"])

    for cc in cond:
        #define  odd and even exp file
        #note that it should be pulled from the opposite test runs (from even ROI pull odd data)
        #all naming convetions are relative to the ROI that data are being pulled
        #odd_exp_file is even data pulled from *odd* run ROI
        odd_exp_file = f'{results_dir}/{lr}{rois[rr]}_all_even_{cc}.txt'
        even_exp_file = f'{results_dir}/{lr}{rois[rr]}_all_odd_{cc}.txt'

        #load each file and append localizer functional value
        odd_exp = pd.read_csv(odd_exp_file, sep="  ", header=None, names = ["x", "y", "z", f'{cc}_odd'])
        even_exp = pd.read_csv(even_exp_file, sep="  ", header=None, names = ["x", "y", "z", f'{cc}_even'])

        loc_df = loc_df.join([odd_exp[cc + "_odd"],even_exp[cc+"_even"]])
        #sort  by localizer value
        loc_df = loc_df.sort_values(by =['loc'], ascending=False)
        loc_df= loc_df.reset_index(drop=True)

        #demean columns by condition
        row_mean=loc_df.iloc[:,4:loc_df.shape[1]].mean(axis=1)
        #loc_df.iloc[:,4:loc_df.shape[1]] =loc_df.iloc[:,4:loc_df.shape[1]].sub(row_mean,axis=0)
        #test =loc_df.iloc[:,4:loc_df.shape[1]].sub(row_mean,axis=0)

    #Start within-between analysis
    n = 1
    df = pd.DataFrame()
    between_temp = pd.DataFrame() 
    for c1 in cond:
        for c2 in cond:

            temp_x = loc_df[f'{c1}_odd']
            temp_y = loc_df[f'{c2}_even']
            temp = temp_x.rolling(bin_size).corr(temp_y)
            temp = temp.dropna()
            temp= temp.reset_index(drop=True)

            if c1 == c2:
                temp = pd.DataFrame(temp)
                temp.columns= [f'{c1}']
                if df.empty:
                    df = temp
                else:
                    df = df.join(temp)
            else:
                if between_temp.empty:
                    between_temp =temp
                else:
                    between_temp =between_temp + temp
                    n = n + 1

    between = pd.DataFrame(between_temp/n)
    between.columns = ['between']
    df = df.join(between)


    #df = df.sub(between,axis=0)
    return df


Make VSF plots

In [37]:
def plot_data(df, cond, cond_name, y_ax,suf):
    df = df[cond]
    df.columns = cond_name
    ax = df.plot.line()
    ax.set_xlabel("Number of Voxels")
    ax.set_ylabel(y_ax)
    plt.title(f'{lr}{rois[rr]}')
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    plt.savefig(f'{sub_dir}/results/{lr}{rois[rr]}_{suf}.png')
    #plt.close()

In [97]:
for ss in subj_list:
    sub_dir = f"{exp_dir}/sub-{ss}/ses-01/derivatives"
    results_dir = f'{sub_dir}/beta/adaptation'
    roi_dir= f'{sub_dir}/rois'
       
    if os.path.exists(results_dir):
        shutil.rmtree(results_dir)
    
    os.makedirs(results_dir) 
    

    for rr in range(0,len(rois)): #loop across rois
        for lr in ['l','r']: #loop across left and right hemispheres
            roi_file = f'{sub_dir}/rois/{lr}{rois[rr]}{suf}.nii.gz'
                      
            #check if odd and even roi exist
            if os.path.exists(roi_file):            
                print(ss, lr,rois[rr],bin_size)
                
                if bool_extract_roi == True:
                    extract_data(sub_dir, results_dir, lr, rr)
                
                if bool_calc_act == True:
                    # Sort and combine split runs                
                    df = calc_activation(lr, rr, bin_size)
                
                    if len(df) > 2500: 
                        df = df.head(2500)
                    plot_data(df, cond, cond_name, 'beta','adapt')
                
                if bool_calc_mvpa == True:
                    df = calc_mvpa(lr, rr, bin_size)
                    if len(df) > 2500: 
                        df = df.head(2500)

                    plot_data(df,cond + ['between'],cond_name + ['Between'],'Correlation','mvpa')


docnet1001 r V3ab 100
docnet1001 r PPC 100
docnet1001 r APC 100
docnet1001 l LO 100
docnet1002 l V3ab 100
docnet1002 r V3ab 100
docnet1002 l PPC 100
docnet1002 r PPC 100
docnet1002 l APC 100
docnet1002 r APC 100
docnet1002 l V4 100
docnet1002 r V4 100
docnet1002 l LO 100
docnet1002 r LO 100
docnet1002 l PFS 100


In [33]:
ss="docnet1001"
sub_dir = f"{exp_dir}/sub-{ss}/ses-01/derivatives"
results_dir = f'{sub_dir}/beta/adaptation'
roi_dir= f'{sub_dir}/rois'

lr="r"
rr=2
#extract_data(sub_dir, results_dir, lr, rr)

df = calc_activation(lr, rr, bin_size)